In [ ]:
import datetime

import numpy as np
from matplotlib import pyplot as plt

import statfin
import pandas_datareader

In [ ]:
def date(t):
    if isinstance(t, str):
        return datetime.datetime.strptime(t, "%YM%m")
    else:
        return np.array([date(e) for e in t])

In [ ]:
db = statfin.StatFin()

In [ ]:
tbl_prices = db.StatFin.ehi.ge
df_prices = tbl_prices.query(Tiedot="hinta")()

In [ ]:
t = df_prices[df_prices.Polttoneste == "A"].Kuukausi
a = df_prices[df_prices.Polttoneste == "A"].hinta / 100
b = df_prices[df_prices.Polttoneste == "B"].hinta / 100
t = date(t)

t = t[-96:]
a = a[-96:]
b = b[-96:]

In [ ]:
df = pandas_datareader.data.DataReader("DCOILBRENTEU", "fred", t[0], datetime.datetime.now())
brent = df.resample("ME").mean().DCOILBRENTEU.to_numpy()[:len(t)]

In [ ]:
df = pandas_datareader.data.DataReader("DEXUSEU", "fred", t[0], datetime.datetime.now())
eurusd = df.resample("ME").mean().DEXUSEU.to_numpy()[:len(t)]

In [ ]:
c = brent * eurusd / 158.987

In [ ]:
plt.figure(figsize=(10, 4))
ax = plt.gca()
ax.plot(t, a, color="forestgreen", label="E95")
ax.plot(t, b, color="firebrick", label="Diesel")
ax.plot(t, c, color="#444", label="Raakaöljy (Brent)")

ax.axvspan(datetime.datetime(2015, 5, 29), datetime.datetime(2019, 6, 6), color="green", alpha=0.1)
ax.axvspan(datetime.datetime(2019, 6, 6), datetime.datetime(2023, 6, 20), color="red", alpha=0.1)
ax.axvspan(datetime.datetime(2023, 6, 20), datetime.datetime.now(), color="blue", alpha=0.1)

ax.axvline(datetime.datetime(2020, 3, 17), color="goldenrod", linewidth=2, alpha=0.5)
ax.axvline(datetime.datetime(2022, 2, 24), color="red", linewidth=2, alpha=0.5)

ax.set_ylim(0, None)
ax.set_xlim(t[0], t[-1])
ax.set_ylabel("EUR / L")
#ax.grid(True)
ax.legend()

plt.savefig("polttoaine.png", dpi=150, bbox_inches="tight")